## Imports

In [815]:
import yfinance as yf
import pandas as pd  
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
from datetime import datetime, date, timedelta
import os
import json

## Get names of all tickers 

In [816]:
with open("company_tickers.json", mode="r") as ticker_file:
    file_contents = ticker_file.read()
stocks_json = json.loads(file_contents)

# Set up and define app layout

In [817]:
external_stylesheets = ["https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css"]
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Header

In [818]:
header = html.H1("My Stocks", style={"display": "flex", "justify-content": "center"})

# Select date


In [819]:
today = date.today() 
monday = today - timedelta(days=today.weekday()) 
friday = monday + timedelta(days=4)  
weekends = [dt for dt in pd.date_range(date(1900, 1, 1), date.today()) if dt.weekday() >= 5]

date_selector = html.Div([
    dcc.DatePickerRange(
        id="date-selector",
        min_date_allowed =date(1900, 1, 1),
        max_date_allowed =today,
        start_date=monday,
        end_date=friday,
        initial_visible_month=today,
        display_format = "MM-DD-YYYY",
        disabled_days = weekends
    ),
    html.Div(id="date-selector-output")
])

In [820]:
@app.callback(  
    Output("date-selector-output", "children"),
    Input("date-selector", "start_date"),
    Input("date-selector", "end_date")
) 
def update_time(start_date, end_date): 
    if start_date and end_date:
        if start_date == end_date:
            return "Start date and End date have to be different!"
        return 
    return "Please select a date to proceed!"

## Select Interval

In [821]:
interval_selector = html.Div([
    dcc.RadioItems(
        id="interval-selector",
        options = [],
        value="1m",
        style={"padding": "10px"}
    ),
    html.Div(id="interval-selector-output")
],
style={
    "padding": "10px",
    "border": "1px solid #ccc",
    "border-radius": "5px",
    "display": "flex",
    "justify-content": "space-between",
    "width": "81%"
})

In [822]:
# Define default options for intervals
INTERVAL_OPTIONS = [
    {"label": "1 minute", "value": "1m"},
    {"label": "2 minutes", "value": "2m"},
    {"label": "5 minutes", "value": "5m"},
    {"label": "15 minutes", "value": "15m"},
    {"label": "30 minutes", "value": "30m"},
    {"label": "60 minutes", "value": "60m"},
    {"label": "90 minutes", "value": "90m"},
    {"label": "1 day", "value": "1d"},
    {"label": "5 days", "value": "5d"},
    {"label": "1 week", "value": "1wk"},
    {"label": "1 month", "value": "1mo"},
    {"label": "3 months", "value": "3mo"}
]

# Define interval filters based on timeframe and yfinance data provision
INTERVAL_FILTERS = {
    (1, 1): ["1m", "2m", "5m", "15m", "30m", "60m", "90m"],
    (2, 5): ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1d"],
    (6, 7): ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1d"],
    (8, 30): ["2m", "5m", "15m", "30m", "60m", "90m", "1d", "5d", "1wk"],
    (31, 60): ["2m", "5m", "15m", "30m", "60m", "90m", "1d", "5d", "1wk", "1mo"],
    (61, 90): ["1d", "5d", "1wk", "1mo"],
    (91, float('inf')): ["1d", "5d", "1wk", "1mo", "3mo"]
}

def filter_intervals(timeframe):
    for (lower, upper), values in INTERVAL_FILTERS.items():
        if lower <= timeframe <= upper:
            return [opt for opt in INTERVAL_OPTIONS if opt["value"] in values]
    return []

@app.callback(
    Output("interval-selector-output", "children"),
    Output("interval-selector", "options"),
    Output("interval-selector", "value"),
    Input("date-selector", "start_date"),
    Input("date-selector", "end_date")
)
def update_interval(start_date, end_date):
    if not start_date or not end_date:
        return "Please select both start and end dates", [], None

    start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    timeframe = (end_date_dt - start_date_dt).days
    options = filter_intervals(timeframe)
    default_value = options[0]["value"] if options else None

    return "", options, default_value

## Selector indicator

In [823]:
indicator_selector = html.Div([
    dcc.RadioItems(
        id="indicator-selector",
        options=[
            {"label": "Open", "value": "Open"},
            {"label": "High", "value": "High"},
            {"label": "Low", "value": "Low"},
            {"label": "Close", "value": "Close"},
            {"label": "Adjusted Close", "value": "Adj Close"},
            {"label": "Volume", "value": "Volume"}
        ],
        value="Close", # Default
    ),
    html.Div(id="indicator-selector-output")
],
style={
    "padding": "10px",
    "border": "1px solid #ccc",
    "border-radius": "5px",
    "display": "flex",
    "width": "81%"
})

In [824]:
@app.callback(  
    Output("indicator-selector-output", "children"),
    Input("indicator-selector", "value"),
) 
def update_indicator(indicator): 
    if indicator:
        return 
    return "Please select an indicator to proceed!"

## Selector stock

In [825]:
options = [
    {"label": stock["title"], "value": stock["ticker"]}
    for stock in list(stocks_json.values())
]

stock_selector = html.Div([
    dcc.Dropdown(
        id="stock-selector",
        options=options,
        multi=True,
        searchable=True,
        clearable=True,
        placeholder="Select a stock",
        style = {"width": "90%"}
    ),
    html.Button("Submit", id="submit-button", n_clicks=0, className="btn btn-primary mt-3"),
    html.Div(id="stock-selector-output")
])

## Download data for selected data in the selected timeframe, and plot candlestick charts of the data in accordance to the selected ticker.

In [826]:
@app.callback(  
    Output("stock-selector-output", "children"),
    Input("stock-selector", "value"),
    Input("indicator-selector", "value"),
    Input("interval-selector", "value"),
    Input("submit-button", "n_clicks"),
    Input("date-selector", "start_date"),
    Input("date-selector", "end_date"),
) 
def update_stocks(stocks, indicator, interval, n_clicks, start_date, end_date):
    if n_clicks > 0:
        if not stocks:
            return html.Div([
                html.Br(),
                html.H1("No stocks selected😢"),
                html.Br(),
                html.H1("Please select at least one stock to proceed!")
            ])
        
        stocks_df = pd.DataFrame()
        candlestick_figures = []
        comparison_fig = go.Figure()
        
        for stock in stocks:
            try:
                # Download stock data
                data = yf.download(tickers=stock, start=start_date, end=end_date, interval=interval)
                
                # Add a column to identify the ticker symbol
                data["ticker"] = stock
                
                # Append the data to the DataFrame
                stocks_df = pd.concat([stocks_df, data])
                
                # Create a trace for the selected indicator
                comparison_fig.add_trace(go.Scatter(
                    x=data.index,
                    y=data[indicator],
                    mode="lines+markers",
                    name=stock
                ))
                
                # Generate a candle stick for each stock
                fig = go.Figure(data=[go.Candlestick(
                        x=data.index,
                        open=data["Open"],
                        high=data["High"],
                        low=data["Low"],
                        close=data["Close"],
                        name=stock)])
                
                fig.update_layout(
                    title={
                        'text': f'<b>Candlestick Chart for <i>{indicator}</i> of {stock} from {start_date} to {end_date}</b>',
                        'y': 0.9,  
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                    },
                    xaxis_title='Date',
                    yaxis_title='Price'
                )
                candlestick_figures.append(dcc.Graph(figure=fig))

            except Exception as e:
                print(f"Error downloading data for {stock}: {e}")
                continue
           
        comparison_fig.update_layout(
            title=f'Comparison of selected stocks based on <b>{indicator}</b>',
            xaxis_title='Date',
            yaxis_title=indicator
        )
        
        return html.Div([
             html.H4(f"Data for {' | '.join(stocks)}"),
             dcc.Graph(figure=comparison_fig),
             *candlestick_figures,
        ])
    
    return "Select time range, indicator, interval, and stocks, and press Submit."

## Run app

In [827]:
app.layout = html.Div([
    header,
    date_selector,
    indicator_selector,
    interval_selector,
    stock_selector
],
style={
    "margin-left": '80px', 
    "padding": '50px',
    "display": "flex", 
    "flex-direction": "column",
    "gap": "2px"  
})

# Run app
app.run(jupyter_mode="external", debug_mode=True)

Dash app running on http://127.0.0.1:8050/
